In [719]:
using LowLevelFEM
import LowLevelFEM as FEM

gmsh.initialize()

In [720]:
gmsh.open("body.geo")

Info    : Reading 'body.geo'...
Info    : Meshing 1D...                                                                                                     
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 20%] Meshing curve 4 (Line)
Info    : [ 30%] Meshing curve 5 (Line)
Info    : [ 30%] Meshing curve 6 (Line)
Info    : [ 40%] Meshing curve 7 (Line)
Info    : [ 40%] Meshing curve 8 (Line)
Info    : [ 50%] Meshing curve 9 (Line)
Info    : [ 50%] Meshing curve 10 (Line)
Info    : [ 60%] Meshing curve 11 (Line)
Info    : [ 60%] Meshing curve 12 (Line)
Info    : [ 70%] Meshing curve 13 (Line)
Info    : [ 70%] Meshing curve 14 (Line)
Info    : [ 80%] Meshing curve 15 (Line)
Info    : [ 80%] Meshing curve 16 (Line)
Info    : [ 90%] Meshing curve 17 (Line)
Info    : [ 90%] Meshing curve 18 (Line)
Info    : [100%] Meshing curve 19 (Line)
Info    : [100%] Meshing curve 20 (Line)
Info    : Done meshing 1D (Wall 

In [721]:
mat1 = FEM.material("body1")
mat2 = FEM.material("body2")
problem = FEM.Problem([mat1, mat2])

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 61)


LowLevelFEM.Problem("body", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body1", 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5), LowLevelFEM.Material("body2", 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5)], 1.0, 8425)

In [722]:
function nodePositionVector(problem)
    dim = problem.dim
    non = problem.non
    if dim != 3
        error("nodePositionVector: This function only works for 3D problems.")
    end
    r = zeros(non * dim)
    for ipg in 1:length(problem.material)
        phName = problem.material[ipg].phName
        dimTags = gmsh.model.getEntitiesForPhysicalName(phName)
        for idm in 1:length(dimTags)
            dimTag = dimTags[idm]
            edim = dimTag[1]
            etag = dimTag[2]
            elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(edim, etag)
            nodeTags, ncoord, parametricCoord = gmsh.model.mesh.getNodes(dim, -1, true, false)
            r[nodeTags*3 .- 2] = ncoord[1:3:length(ncoord)]
            r[nodeTags*3 .- 1] = ncoord[2:3:length(ncoord)]
            r[nodeTags*3 .- 0] = ncoord[3:3:length(ncoord)]
        end
    end
    return r
end

r = nodePositionVector(problem);

In [723]:
using SparseArrays

function stiffnessMatrixLinear(problem, r)
    gmsh.model.setCurrent(problem.name)
    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(problem.dim, -1)
    lengthOfIJV = sum([(div(length(elemNodeTags[i]), length(elemTags[i])) * problem.dim)^2 * length(elemTags[i]) for i in 1:length(elemTags)])
    nn = []
    I = []
    J = []
    V = []
    V = convert(Vector{Float64}, V)
    sizehint!(I, lengthOfIJV)
    sizehint!(J, lengthOfIJV)
    sizehint!(V, lengthOfIJV)

    for ipg in 1:length(problem.material)
        phName = problem.material[ipg].phName
        E = problem.material[ipg].E
        ν = problem.material[ipg].ν
        dim = problem.dim
        pdim = problem.pdim
        if problem.dim == 3 && problem.type == :Solid
            D = E / ((1 + ν) * (1 - 2ν)) * [1-ν ν ν 0 0 0;
                ν 1-ν ν 0 0 0;
                ν ν 1-ν 0 0 0;
                0 0 0 (1-2ν)/2 0 0;
                0 0 0 0 (1-2ν)/2 0;
                0 0 0 0 0 (1-2ν)/2]

            rowsOfB = 6
            b = 1
        elseif problem.dim == 2 && problem.type == :PlaneStress
            D = E / (1 - ν^2) * [1 ν 0;
                ν 1 0;
                0 0 (1-ν)/2]
            rowsOfB = 3
            b = problem.thickness
        elseif problem.dim == 2 && problem.type == :PlaneStrain
            D = E / ((1 + ν) * (1 - 2ν)) * [1-ν ν 0;
                ν 1-ν 0;
                0 0 (1-2ν)/2]
            rowsOfB = 3
            b = 1
        else
            error("stiffnessMatrixSolid: dimension is $(problem.dim), problem type is $(problem.type).")
        end

        dimTags = gmsh.model.getEntitiesForPhysicalName(phName)
        for idm in 1:length(dimTags)
            dimTag = dimTags[idm]
            edim = dimTag[1]
            etag = dimTag[2]
            elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(edim, etag)
            for i in 1:length(elemTypes)
                et = elemTypes[i]
                elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
                intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))
                numIntPoints = length(intWeights)
                #comp, fun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                #h = reshape(fun, :, numIntPoints)
                comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, numIntPoints)
                nnet = zeros(Int, length(elemTags[i]), numNodes)
                invJac = zeros(3, 3numIntPoints)
                Iidx = zeros(Int, numNodes * pdim, numNodes * pdim)
                Jidx = zeros(Int, numNodes * pdim, numNodes * pdim)
                for k in 1:numNodes*pdim, l in 1:numNodes*pdim
                    Iidx[k, l] = l
                    Jidx[k, l] = k
                end
                #H = zeros(9 * numIntPoints, pdim * numNodes)
                ∂h = zeros(dim, numNodes * numIntPoints)
                B = zeros(rowsOfB * numIntPoints, pdim * numNodes)
                ∂H = spzeros(9 * numIntPoints, pdim * numNodes)
                K1 = zeros(pdim * numNodes, pdim * numNodes)
                nn2 = zeros(Int, pdim * numNodes)
                sizehint!(∂H, 9 * numNodes)
                #for k in 1:numIntPoints, l in 1:numNodes
                #    for kk in 1:3:9, ll in 1:3
                #H[k*9-(9-kk)-1+ll, l*pdim-(pdim-ll)] = h[(k-1)*numNodes+l]
                #    end
                #end
                for j in 1:length(elemTags[i])
                    elem = elemTags[i][j]
                    for k in 1:numNodes
                        nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
                    end
                    for k in 1:pdim
                        nn2[k:pdim:pdim*numNodes] = pdim * nnet[j, 1:numNodes] .- (pdim - k)
                    end
                    jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)
                    for k in 1:numIntPoints
                        invJac[1:3, 3*k-2:3*k] = @inline inv(Jac[1:3, 3*k-2:3*k])'
                    end
                    ∂h .*= 0
                    for k in 1:numIntPoints, l in 1:numNodes
                        ∂h[1:dim, (k-1)*numNodes+l] .= invJac[1:dim, k*3-2:k*3-(3-dim)] * ∇h[l*3-2:l*3-(3-dim), k]
                    end
                    r1 = r[nn2]
                    B .*= 0
                    ∂H .*= 0
                    if dim == 2 && rowsOfB == 3
                        for k in 1:numIntPoints
                            for l in 1:numNodes
                                B[k*rowsOfB-0, l*pdim-0] = B[k*rowsOfB-2, l*pdim-1] = ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-0, l*pdim-1] = B[k*rowsOfB-1, l*pdim-0] = ∂h[2, (k-1)*numNodes+l]
                            end
                        end
                    elseif dim == 3 && rowsOfB == 6
                        for k in 1:numIntPoints
                            for l in 1:numNodes
                                ∂H[k*9-(9-1), l*pdim-(pdim-1)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-2), l*pdim-(pdim-1)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-3), l*pdim-(pdim-1)] = ∂h[3, (k-1)*numNodes+l]
                                ∂H[k*9-(9-4), l*pdim-(pdim-2)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-5), l*pdim-(pdim-2)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-6), l*pdim-(pdim-2)] = ∂h[3, (k-1)*numNodes+l]
                                ∂H[k*9-(9-7), l*pdim-(pdim-3)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-8), l*pdim-(pdim-3)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-9), l*pdim-(pdim-3)] = ∂h[3, (k-1)*numNodes+l]
                            end
                            ∂H1 = ∂H[k*9-(9-1):k*9, 1:pdim*numNodes]
                            F = ∂H1 * r1
                            for l in 1:numNodes
                                B[k*rowsOfB-5, l*pdim-2] = F[1] * ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-4, l*pdim-2] = F[4] * ∂h[2, (k-1)*numNodes+l]
                                B[k*rowsOfB-3, l*pdim-2] = F[7] * ∂h[3, (k-1)*numNodes+l]
                                B[k*rowsOfB-2, l*pdim-2] = (F[4] * ∂h[1, (k-1)*numNodes+l] + F[1] * ∂h[2, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-1, l*pdim-2] = (F[7] * ∂h[2, (k-1)*numNodes+l] + F[4] * ∂h[3, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-0, l*pdim-2] = (F[1] * ∂h[3, (k-1)*numNodes+l] + F[7] * ∂h[1, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-5, l*pdim-1] = F[2] * ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-4, l*pdim-1] = F[5] * ∂h[2, (k-1)*numNodes+l]
                                B[k*rowsOfB-3, l*pdim-1] = F[8] * ∂h[3, (k-1)*numNodes+l]
                                B[k*rowsOfB-2, l*pdim-1] = (F[5] * ∂h[1, (k-1)*numNodes+l] + F[2] * ∂h[2, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-1, l*pdim-1] = (F[8] * ∂h[2, (k-1)*numNodes+l] + F[5] * ∂h[3, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-0, l*pdim-1] = (F[2] * ∂h[3, (k-1)*numNodes+l] + F[8] * ∂h[1, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-5, l*pdim-0] = F[3] * ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-4, l*pdim-0] = F[6] * ∂h[2, (k-1)*numNodes+l]
                                B[k*rowsOfB-3, l*pdim-0] = F[9] * ∂h[3, (k-1)*numNodes+l]
                                B[k*rowsOfB-2, l*pdim-0] = (F[6] * ∂h[1, (k-1)*numNodes+l] + F[3] * ∂h[2, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-1, l*pdim-0] = (F[9] * ∂h[2, (k-1)*numNodes+l] + F[6] * ∂h[3, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-0, l*pdim-0] = (F[1] * ∂h[3, (k-1)*numNodes+l] + F[9] * ∂h[1, (k-1)*numNodes+l]) / 2
                            end
                        end
                    else
                        error("stiffnessMatrixLinear: rows of B is $rowsOfB, dimension of the problem is $dim.")
                    end
                    K1 .*= 0
                    for k in 1:numIntPoints
                        B1 = B[k*rowsOfB-(rowsOfB-1):k*rowsOfB, 1:pdim*numNodes]
                        K1 += B1' * D * B1 * b * jacDet[k] * intWeights[k]
                    end
                    append!(I, nn2[Iidx[:]])
                    append!(J, nn2[Jidx[:]])
                    append!(V, K1[:])
                end
                push!(nn, nnet)
            end
        end
    end
    dof = problem.pdim * problem.non
    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return K
end

stiffnessMatrixLinear (generic function with 1 method)

In [724]:
using SparseArrays

function stiffnessMatrixNonLinear(problem, S)
    gmsh.model.setCurrent(problem.name)
    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(problem.dim, -1)
    lengthOfIJV = sum([(div(length(elemNodeTags[i]), length(elemTags[i])) * problem.dim)^2 * length(elemTags[i]) for i in 1:length(elemTags)])
    nn = []
    I = []
    J = []
    V = []
    V = convert(Vector{Float64}, V)
    sizehint!(I, lengthOfIJV)
    sizehint!(J, lengthOfIJV)
    sizehint!(V, lengthOfIJV)

    for ipg in 1:length(problem.material)
        phName = problem.material[ipg].phName
        E = problem.material[ipg].E
        ν = problem.material[ipg].ν
        dim = problem.dim
        pdim = problem.pdim
        if problem.dim == 3 && problem.type == :Solid
            rowsOfB = 6
            b = 1
        elseif problem.dim == 2 && problem.type == :PlaneStress
            rowsOfB = 3
            b = problem.thickness
        elseif problem.dim == 2 && problem.type == :PlaneStrain
            rowsOfB = 3
            b = 1
        else
            error("stiffnessMatrixSolid: dimension is $(problem.dim), problem type is $(problem.type).")
        end

        dimTags = gmsh.model.getEntitiesForPhysicalName(phName)
        for idm in 1:length(dimTags)
            dimTag = dimTags[idm]
            edim = dimTag[1]
            etag = dimTag[2]
            elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(edim, etag)
            for i in 1:length(elemTypes)
                et = elemTypes[i]
                elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
                intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))
                numIntPoints = length(intWeights)
                comp, fun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                h = reshape(fun, :, numIntPoints)
                comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, numIntPoints)
                nnet = zeros(Int, length(elemTags[i]), numNodes)
                invJac = zeros(3, 3numIntPoints)
                Iidx = zeros(Int, numNodes * pdim, numNodes * pdim)
                Jidx = zeros(Int, numNodes * pdim, numNodes * pdim)
                for k in 1:numNodes*pdim, l in 1:numNodes*pdim
                    Iidx[k, l] = l
                    Jidx[k, l] = k
                end
                H = spzeros(9 * numIntPoints, 9 * numNodes)
                sizehint!(H, 9 * numNodes * numIntPoints)
                ∂h = zeros(dim, numNodes * numIntPoints)
                B = zeros(rowsOfB * numIntPoints, pdim * numNodes)
                ∂H = spzeros(9 * numIntPoints, pdim * numNodes)
                K1 = zeros(pdim * numNodes, pdim * numNodes)
                nn2 = zeros(Int, dim * numNodes)
                nn9 = zeros(Int, 9 * numNodes)
                sizehint!(∂H, 9 * numNodes * numIntPoints)
                for k in 1:numIntPoints, l in 1:numNodes
                    for kk in 1:9
                        H[k*9-(9-kk), l*9-(9-kk)] = h[(k-1)*numNodes+l]
                    end
                end
                for j in 1:length(elemTags[i])
                    elem = elemTags[i][j]
                    for k in 1:numNodes
                        nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
                    end
                    for k in 1:dim
                        nn2[k:dim:dim*numNodes] = dim * nnet[j, 1:numNodes] .- (dim - k)
                    end
                    for k in 1:9
                        nn9[k:9:9*numNodes] = 9 * nnet[j, 1:numNodes] .- (9 - k)
                    end
                    jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)
                    for k in 1:numIntPoints
                        invJac[1:3, 3*k-2:3*k] = @inline inv(Jac[1:3, 3*k-2:3*k])'
                    end
                    ∂h .*= 0
                    for k in 1:numIntPoints, l in 1:numNodes
                        ∂h[1:dim, (k-1)*numNodes+l] .= invJac[1:dim, k*3-2:k*3-(3-dim)] * ∇h[l*3-2:l*3-(3-dim), k]
                    end
                    S1 = S[nn9]
                    B .*= 0
                    ∂H .*= 0
                    if dim == 2 && rowsOfB == 3
                        for k in 1:numIntPoints
                            for l in 1:numNodes
                                B[k*rowsOfB-0, l*pdim-0] = B[k*rowsOfB-2, l*pdim-1] = ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-0, l*pdim-1] = B[k*rowsOfB-1, l*pdim-0] = ∂h[2, (k-1)*numNodes+l]
                            end
                        end
                    elseif dim == 3 && rowsOfB == 6
                        for k in 1:numIntPoints
                            for l in 1:numNodes
                                ∂H[k*9-(9-1), l*pdim-(pdim-1)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-2), l*pdim-(pdim-1)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-3), l*pdim-(pdim-1)] = ∂h[3, (k-1)*numNodes+l]
                                ∂H[k*9-(9-4), l*pdim-(pdim-2)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-5), l*pdim-(pdim-2)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-6), l*pdim-(pdim-2)] = ∂h[3, (k-1)*numNodes+l]
                                ∂H[k*9-(9-7), l*pdim-(pdim-3)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-8), l*pdim-(pdim-3)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-9), l*pdim-(pdim-3)] = ∂h[3, (k-1)*numNodes+l]
                            end
                        end
                    else
                        error("stiffnessMatrixLinear: rows of B is $rowsOfB, dimension of the problem is $dim.")
                    end
                    K1 .*= 0
                    for k in 1:numIntPoints
                        H1 = H[k*9-(9-1):k*9, 1:9*numNodes]
                        S0 = reshape(H1 * S1, 3, 3)
                        ∂H1 = ∂H[k*9-(9-1):k*9-(9-3), 1:pdim*numNodes]
                        ∂H2 = ∂H[k*9-(9-4):k*9-(9-6), 1:pdim*numNodes]
                        ∂H3 = ∂H[k*9-(9-7):k*9-(9-9), 1:pdim*numNodes]
                        K1 += (∂H1' * S0 * ∂H1 + ∂H2' * S0 * ∂H2 + ∂H3' * S0 * ∂H3) * b * jacDet[k] * intWeights[k]
                    end
                    append!(I, nn2[Iidx[:]])
                    append!(J, nn2[Jidx[:]])
                    append!(V, K1[:])
                end
                push!(nn, nnet)
            end
        end
    end
    dof = problem.pdim * problem.non
    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return K
end

stiffnessMatrixNonLinear (generic function with 1 method)

In [725]:
using SparseArrays

function loadVectorNonLinear(problem, r, S)
    gmsh.model.setCurrent(problem.name)
    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(problem.dim, -1)
    #lengthOfIJV = sum([(div(length(elemNodeTags[i]), length(elemTags[i])) * problem.dim)^2 * length(elemTags[i]) for i in 1:length(elemTags)])
    #nn = []
    #I = []
    #J = []
    #V = []
    #V = convert(Vector{Float64}, V)
    #sizehint!(I, lengthOfIJV)
    #sizehint!(J, lengthOfIJV)
    #sizehint!(V, lengthOfIJV)
    f = zeros(problem.non * problem.pdim)

    for ipg in 1:length(problem.material)
        phName = problem.material[ipg].phName
        E = problem.material[ipg].E
        ν = problem.material[ipg].ν
        dim = problem.dim
        pdim = problem.pdim
        if problem.dim == 3 && problem.type == :Solid
            D = E / ((1 + ν) * (1 - 2ν)) * [1-ν ν ν 0 0 0;
                ν 1-ν ν 0 0 0;
                ν ν 1-ν 0 0 0;
                0 0 0 (1-2ν)/2 0 0;
                0 0 0 0 (1-2ν)/2 0;
                0 0 0 0 0 (1-2ν)/2]

            rowsOfB = 6
            b = 1
        elseif problem.dim == 2 && problem.type == :PlaneStress
            D = E / (1 - ν^2) * [1 ν 0;
                ν 1 0;
                0 0 (1-ν)/2]
            rowsOfB = 3
            b = problem.thickness
        elseif problem.dim == 2 && problem.type == :PlaneStrain
            D = E / ((1 + ν) * (1 - 2ν)) * [1-ν ν 0;
                ν 1-ν 0;
                0 0 (1-2ν)/2]
            rowsOfB = 3
            b = 1
        else
            error("stiffnessMatrixSolid: dimension is $(problem.dim), problem type is $(problem.type).")
        end

        dimTags = gmsh.model.getEntitiesForPhysicalName(phName)
        for idm in 1:length(dimTags)
            dimTag = dimTags[idm]
            edim = dimTag[1]
            etag = dimTag[2]
            elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(edim, etag)
            for i in 1:length(elemTypes)
                et = elemTypes[i]
                elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
                intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))
                numIntPoints = length(intWeights)
                comp, fun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                h = reshape(fun, :, numIntPoints)
                comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, numIntPoints)
                nnet = zeros(Int, length(elemTags[i]), numNodes)
                invJac = zeros(3, 3numIntPoints)
                Iidx = zeros(Int, numNodes * pdim, numNodes * pdim)
                Jidx = zeros(Int, numNodes * pdim, numNodes * pdim)
                for k in 1:numNodes*pdim, l in 1:numNodes*pdim
                    Iidx[k, l] = l
                    Jidx[k, l] = k
                end
                H = spzeros(9 * numIntPoints, 9 * numNodes)
                sizehint!(H, 9 * numNodes * numIntPoints)
                ∂h = zeros(dim, numNodes * numIntPoints)
                B = zeros(rowsOfB * numIntPoints, pdim * numNodes)
                ∂H = spzeros(9 * numIntPoints, pdim * numNodes)
                f1 = zeros(pdim * numNodes)
                nn2 = zeros(Int, pdim * numNodes)
                nn9 = zeros(Int, 9 * numNodes)
                sizehint!(∂H, 9 * numNodes * numIntPoints)
                #for k in 1:numIntPoints, l in 1:numNodes
                #    for kk in 1:3:9, ll in 1:3
                #H[k*9-(9-kk)-1+ll, l*pdim-(pdim-ll)] = h[(k-1)*numNodes+l]
                #    end
                #end
                for j in 1:length(elemTags[i])
                    elem = elemTags[i][j]
                    for k in 1:numNodes
                        nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
                    end
                    for k in 1:pdim
                        nn2[k:pdim:pdim*numNodes] = pdim * nnet[j, 1:numNodes] .- (pdim - k)
                    end
                    for k in 1:9
                        nn9[k:9:9*numNodes] = 9 * nnet[j, 1:numNodes] .- (9 - k)
                    end
                    jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)
                    for k in 1:numIntPoints
                        invJac[1:3, 3*k-2:3*k] = @inline inv(Jac[1:3, 3*k-2:3*k])'
                    end
                    ∂h .*= 0
                    for k in 1:numIntPoints, l in 1:numNodes
                        ∂h[1:dim, (k-1)*numNodes+l] .= invJac[1:dim, k*3-2:k*3-(3-dim)] * ∇h[l*3-2:l*3-(3-dim), k]
                    end
                    r1 = r[nn2]
                    S1 = S[nn9]
                    B .*= 0
                    ∂H .*= 0
                    if dim == 2 && rowsOfB == 3
                        for k in 1:numIntPoints
                            for l in 1:numNodes
                                B[k*rowsOfB-0, l*pdim-0] = B[k*rowsOfB-2, l*pdim-1] = ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-0, l*pdim-1] = B[k*rowsOfB-1, l*pdim-0] = ∂h[2, (k-1)*numNodes+l]
                            end
                        end
                    elseif dim == 3 && rowsOfB == 6
                        for k in 1:numIntPoints
                            for l in 1:numNodes
                                ∂H[k*9-(9-1), l*pdim-(pdim-1)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-2), l*pdim-(pdim-1)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-3), l*pdim-(pdim-1)] = ∂h[3, (k-1)*numNodes+l]
                                ∂H[k*9-(9-4), l*pdim-(pdim-2)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-5), l*pdim-(pdim-2)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-6), l*pdim-(pdim-2)] = ∂h[3, (k-1)*numNodes+l]
                                ∂H[k*9-(9-7), l*pdim-(pdim-3)] = ∂h[1, (k-1)*numNodes+l]
                                ∂H[k*9-(9-8), l*pdim-(pdim-3)] = ∂h[2, (k-1)*numNodes+l]
                                ∂H[k*9-(9-9), l*pdim-(pdim-3)] = ∂h[3, (k-1)*numNodes+l]
                            end
                            ∂H1 = ∂H[k*9-(9-1):k*9, 1:pdim*numNodes]
                            F = ∂H1 * r1
                            for l in 1:numNodes
                                B[k*rowsOfB-5, l*pdim-2] = F[1] * ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-4, l*pdim-2] = F[4] * ∂h[2, (k-1)*numNodes+l]
                                B[k*rowsOfB-3, l*pdim-2] = F[7] * ∂h[3, (k-1)*numNodes+l]
                                B[k*rowsOfB-2, l*pdim-2] = (F[4] * ∂h[1, (k-1)*numNodes+l] + F[1] * ∂h[2, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-1, l*pdim-2] = (F[7] * ∂h[2, (k-1)*numNodes+l] + F[4] * ∂h[3, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-0, l*pdim-2] = (F[1] * ∂h[3, (k-1)*numNodes+l] + F[7] * ∂h[1, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-5, l*pdim-1] = F[2] * ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-4, l*pdim-1] = F[5] * ∂h[2, (k-1)*numNodes+l]
                                B[k*rowsOfB-3, l*pdim-1] = F[8] * ∂h[3, (k-1)*numNodes+l]
                                B[k*rowsOfB-2, l*pdim-1] = (F[5] * ∂h[1, (k-1)*numNodes+l] + F[2] * ∂h[2, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-1, l*pdim-1] = (F[8] * ∂h[2, (k-1)*numNodes+l] + F[5] * ∂h[3, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-0, l*pdim-1] = (F[2] * ∂h[3, (k-1)*numNodes+l] + F[8] * ∂h[1, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-5, l*pdim-0] = F[3] * ∂h[1, (k-1)*numNodes+l]
                                B[k*rowsOfB-4, l*pdim-0] = F[6] * ∂h[2, (k-1)*numNodes+l]
                                B[k*rowsOfB-3, l*pdim-0] = F[9] * ∂h[3, (k-1)*numNodes+l]
                                B[k*rowsOfB-2, l*pdim-0] = (F[6] * ∂h[1, (k-1)*numNodes+l] + F[3] * ∂h[2, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-1, l*pdim-0] = (F[9] * ∂h[2, (k-1)*numNodes+l] + F[6] * ∂h[3, (k-1)*numNodes+l]) / 2
                                B[k*rowsOfB-0, l*pdim-0] = (F[1] * ∂h[3, (k-1)*numNodes+l] + F[9] * ∂h[1, (k-1)*numNodes+l]) / 2
                            end
                        end
                    else
                        error("stiffnessMatrixLinear: rows of B is $rowsOfB, dimension of the problem is $dim.")
                    end
                    f1 .*= 0
                    for k in 1:numIntPoints
                        H1 = H[k*9-(9-1):k*9, 1:9*numNodes]
                        S0 = H1 * S1
                        B1 = B[k*rowsOfB-(rowsOfB-1):k*rowsOfB, 1:pdim*numNodes]
                        f1 += B1' * S0[[1, 5, 9, 4, 6, 3]] * b * jacDet[k] * intWeights[k]
                    end
                    #append!(I, nn2[Iidx[:]])
                    #append!(J, nn2[Jidx[:]])
                    #append!(V, K1[:])
                end
                #push!(nn, nnet)
                f[nn2] += f1
            end
        end
    end
    #dof = problem.pdim * problem.non
    #K = sparse(I, J, V, dof, dof)
    #dropzeros!(K)
    return f
end

loadVectorNonLinear (generic function with 1 method)

In [726]:
fx(x, y, z) = 1000 * (y - 0.5)
supp = FEM.displacementConstraint("left", ux=0, uy=0, uz=0)
load = FEM.load("right", fx=fx)

q = FEM.solveDisplacement(problem, [load], [supp])
S = FEM.solveStress(problem, q)
S = FEM.elementsToNodes(problem, S);

In [727]:
Kl = stiffnessMatrixLinear(problem, r)

25275×25275 SparseMatrixCSC{Float64, Int64} with 1533762 stored entries:
⎡⡿⣯⡈⠈⠁⠉⢨⢷⣲⡅⠀⠘⠇⡩⣌⠁⠁⠀⠈⠀⠈⠈⠈⠁⠉⠘⠇⡁⠉⠈⢥⡉⠈⠁⠁⠁⠈⠁⠉⠈⎤
⎢⡂⠈⠻⣦⡀⣀⢐⠀⠀⠻⠽⣖⠀⠙⠀⠲⣄⡀⢀⠀⢀⣀⢀⡀⣀⢀⠀⠙⠂⠀⠀⠐⢦⡀⡀⣀⠀⡀⣀⢀⎥
⎢⡅⠀⠀⢨⠻⣦⣨⠀⠀⠀⠀⠀⠀⠀⠀⠀⡝⠛⢯⢤⣀⠉⢠⡳⢄⠀⠀⠀⠀⠀⠀⠀⠀⠁⢤⡀⠀⠙⢆⡀⎥
⎢⢦⣖⠐⠐⠂⠚⠻⣦⡀⠂⠀⢰⠂⠲⡔⠂⠂⠀⠘⠉⠓⠐⠐⠂⠚⢳⡂⠂⡖⠐⢦⠒⠐⠂⠂⠒⠐⠂⠒⠙⎥
⎢⠜⠾⣤⡀⠀⠀⠠⠈⠻⣦⡀⠈⠇⡄⠳⢤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠇⣄⠹⢠⠘⠦⡄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣀⠀⢳⢧⠀⠀⢀⣀⡀⠈⠻⣦⣀⢹⠀⠘⡆⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⢹⡄⠈⡇⠀⢳⠀⠀⠀⠀⠀⠀⠀⎥
⎢⡍⡡⣄⠀⠀⠀⢨⡀⠉⠥⣄⣘⢿⣷⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⢿⣷⡄⠀⡀⡀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠆⠙⢠⡀⠀⠀⠰⠉⠙⣆⣀⠀⠀⠘⢿⣷⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣿⡦⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠁⠀⠀⠹⣷⠉⠈⠀⠀⠀⠈⠉⠀⠀⠀⠉⠻⣦⡈⠀⠈⣏⠈⢳⠉⠈⠀⠀⠀⠀⠀⠈⠹⡅⠁⣏⠈⢳⠉⠈⎥
⎢⠂⠀⠀⠐⠋⣗⡖⠀⠀⠀⠀⠀⠀⠀⠀⠀⠂⠈⠻⣦⡀⠘⢰⠈⠳⡆⠀⠀⠀⠀⠀⠀⠀⠃⢦⠘⢰⡈⠓⡆⎥
⎢⡂⠀⠀⢰⡄⠘⢙⠀⠀⠀⠀⠀⠀⠀⠀⠀⡦⢤⣀⠈⢻⣶⣌⠃⠀⠛⠀⠀⠀⠀⠀⠀⢰⣶⣜⠂⠀⠃⠀⠙⎥
⎢⠆⠀⠀⠰⢤⡲⠰⠀⠀⠀⠀⠀⠀⠀⠀⠀⢦⣀⡐⠒⠦⠙⠿⣧⣄⠀⠀⠀⠀⠀⠀⠀⠈⠻⠿⠇⠀⢤⣄⠀⎥
⎢⣃⠀⠀⢘⠀⠑⢾⣀⠀⠀⠀⢀⣀⠀⠀⠀⡃⠀⠹⠦⣤⠀⠀⠙⢿⣷⣀⠀⣀⡀⠀⠀⠀⠀⠀⠀⠀⠙⢿⢷⎥
⎢⠍⠡⣄⠀⠀⠀⠨⠈⠉⢥⣄⣘⢿⣷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⢿⣷⡿⣿⡆⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⡃⠀⠈⠀⠀⠀⢘⠉⠓⣂⡀⠉⠀⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠸⣿⣯⣿⣿⡿⣿⣦⠀⠀⠀⠀⠀⠀⠀⎥
⎢⡅⠳⢀⠀⠀⠀⢨⠓⠲⡄⠉⠉⠀⠨⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉⣿⣯⣿⣿⣍⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠆⠀⠈⠳⠄⠀⠰⠀⠀⠉⠙⠒⠀⠀⠈⠋⠗⠦⠤⠀⢰⣶⣦⡀⠀⠀⠀⠀⠈⠛⠃⠙⢻⣶⣄⣶⣦⡀⠀⠀⎥
⎢⠅⠀⠀⢨⠀⠳⢨⠀⠀⠀⠀⠀⠀⠀⠀⠀⡥⢤⣈⠓⠲⠙⠿⠇⠀⠀⠀⠀⠀⠀⠀⠀⢠⣽⡿⣯⣿⣧⣤⡀⎥
⎢⠆⠀⠀⠠⣄⠀⠰⠀⠀⠀⠀⠀⠀⠀⠀⠀⢦⣀⡐⠲⠤⠀⠀⣄⣄⠀⠀⠀⠀⠀⠀⠀⠈⠻⠿⣿⡿⣯⣿⠿⎥
⎣⡃⠀⠀⢘⠈⠱⣜⠀⠀⠀⠀⠀⠀⠀⠀⠀⡃⠀⠹⠤⣄⠀⠀⠙⢿⣗⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣿⡟⢿⣷⎦

In [728]:
Knl = stiffnessMatrixNonLinear(problem, S)

25275×25275 SparseMatrixCSC{Float64, Int64} with 511563 stored entries:
⎡⡿⣯⡈⠈⠁⠉⢨⢷⣲⡅⠀⠘⠇⡩⣌⠁⠁⠀⠈⠀⠈⠈⠈⠁⠉⠘⠇⡁⠉⠈⢥⡉⠈⠁⠁⠁⠈⠁⠉⠈⎤
⎢⡂⠈⠻⣦⡀⣀⢐⠀⠀⠻⠽⣖⠀⠙⠀⠲⣄⡀⢀⠀⢀⣀⢀⡀⣀⢀⠀⠙⠂⠀⠀⠐⢦⡀⡀⣀⠀⡀⣀⢀⎥
⎢⡅⠀⠀⢨⠻⣦⣨⠀⠀⠀⠀⠀⠀⠀⠀⠀⡝⠛⢯⢤⣀⠉⢠⡳⢄⠀⠀⠀⠀⠀⠀⠀⠀⠁⢤⡀⠀⠙⢆⡀⎥
⎢⢦⣖⠐⠐⠂⠚⠻⣦⡀⠂⠀⢰⠂⠲⡔⠂⠂⠀⠘⠉⠓⠐⠐⠂⠚⢳⡂⠂⡖⠐⢦⠒⠐⠂⠂⠒⠐⠂⠒⠙⎥
⎢⠜⠾⣤⡀⠀⠀⠠⠈⠻⣦⡀⠈⠇⡄⠳⢤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠇⣄⠹⢠⠘⠦⡄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣀⠀⢳⢧⠀⠀⢀⣀⡀⠈⠻⣦⣀⢹⠀⠘⡆⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⢹⡄⠈⡇⠀⢳⠀⠀⠀⠀⠀⠀⠀⎥
⎢⡍⡡⣄⠀⠀⠀⢨⡀⠉⠥⣄⣘⢿⣷⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⢿⣷⡄⠀⡀⡀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠆⠙⢠⡀⠀⠀⠰⠉⠙⣆⣀⠀⠀⠘⢿⣷⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣿⡦⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠁⠀⠀⠹⣷⠉⠈⠀⠀⠀⠈⠉⠀⠀⠀⠉⠻⣦⡈⠀⠈⣏⠈⢳⠉⠈⠀⠀⠀⠀⠀⠈⠹⡅⠁⣏⠈⢳⠉⠈⎥
⎢⠂⠀⠀⠐⠋⣗⡖⠀⠀⠀⠀⠀⠀⠀⠀⠀⠂⠈⠻⣦⡀⠘⢰⠈⠳⡆⠀⠀⠀⠀⠀⠀⠀⠃⢦⠘⢰⡈⠓⡆⎥
⎢⡂⠀⠀⢰⡄⠘⢙⠀⠀⠀⠀⠀⠀⠀⠀⠀⡦⢤⣀⠈⢻⣶⣌⠃⠀⠛⠀⠀⠀⠀⠀⠀⢰⣶⣜⠂⠀⠃⠀⠙⎥
⎢⠆⠀⠀⠰⢤⡲⠰⠀⠀⠀⠀⠀⠀⠀⠀⠀⢦⣀⡐⠒⠦⠙⠿⣧⣄⠀⠀⠀⠀⠀⠀⠀⠈⠻⠿⠇⠀⢤⣄⠀⎥
⎢⣃⠀⠀⢘⠀⠑⢾⣀⠀⠀⠀⢀⣀⠀⠀⠀⡃⠀⠹⠦⣤⠀⠀⠙⢿⣷⣀⠀⣀⡀⠀⠀⠀⠀⠀⠀⠀⠙⢿⢷⎥
⎢⠍⠡⣄⠀⠀⠀⠨⠈⠉⢥⣄⣘⢿⣷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⢿⣷⡿⣿⡆⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⡃⠀⠈⠀⠀⠀⢘⠉⠓⣂⡀⠉⠀⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠸⣿⣯⣿⣿⡿⣿⣦⠀⠀⠀⠀⠀⠀⠀⎥
⎢⡅⠳⢀⠀⠀⠀⢨⠓⠲⡄⠉⠉⠀⠨⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉⣿⣯⣿⣿⣍⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠆⠀⠈⠳⠄⠀⠰⠀⠀⠉⠙⠒⠀⠀⠈⠋⠗⠦⠤⠀⢰⣶⣦⡀⠀⠀⠀⠀⠈⠛⠃⠙⢻⣶⣄⣶⣦⡀⠀⠀⎥
⎢⠅⠀⠀⢨⠀⠳⢨⠀⠀⠀⠀⠀⠀⠀⠀⠀⡥⢤⣈⠓⠲⠙⠿⠇⠀⠀⠀⠀⠀⠀⠀⠀⢠⣽⡿⣯⣿⣧⣤⡀⎥
⎢⠆⠀⠀⠠⣄⠀⠰⠀⠀⠀⠀⠀⠀⠀⠀⠀⢦⣀⡐⠲⠤⠀⠀⣄⣄⠀⠀⠀⠀⠀⠀⠀⠈⠻⠿⣿⡿⣯⣿⠿⎥
⎣⡃⠀⠀⢘⠈⠱⣜⠀⠀⠀⠀⠀⠀⠀⠀⠀⡃⠀⠹⠤⣄⠀⠀⠙⢿⣗⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣿⡟⢿⣷⎦

In [729]:
K = FEM.stiffnessMatrix(problem)

25275×25275 SparseMatrixCSC{Float64, Int64} with 1533610 stored entries:
⎡⡿⣯⡈⠈⠁⠉⢨⢷⣲⡅⠀⠘⠇⡩⣌⠁⠁⠀⠈⠀⠈⠈⠈⠁⠉⠘⠇⡁⠉⠈⢥⡉⠈⠁⠁⠁⠈⠁⠉⠈⎤
⎢⡂⠈⠻⣦⡀⣀⢐⠀⠀⠻⠽⣖⠀⠙⠀⠲⣄⡀⢀⠀⢀⣀⢀⡀⣀⢀⠀⠙⠂⠀⠀⠐⢦⡀⡀⣀⠀⡀⣀⢀⎥
⎢⡅⠀⠀⢨⠻⣦⣨⠀⠀⠀⠀⠀⠀⠀⠀⠀⡝⠛⢯⢤⣀⠉⢠⡳⢄⠀⠀⠀⠀⠀⠀⠀⠀⠁⢤⡀⠀⠙⢆⡀⎥
⎢⢦⣖⠐⠐⠂⠚⠻⣦⡀⠂⠀⢰⠂⠲⡔⠂⠂⠀⠘⠉⠓⠐⠐⠂⠚⢳⡂⠂⡖⠐⢦⠒⠐⠂⠂⠒⠐⠂⠒⠙⎥
⎢⠜⠾⣤⡀⠀⠀⠠⠈⠻⣦⡀⠈⠇⡄⠳⢤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠇⣄⠹⢠⠘⠦⡄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣀⠀⢳⢧⠀⠀⢀⣀⡀⠈⠻⣦⣀⢹⠀⠘⡆⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⢹⡄⠈⡇⠀⢳⠀⠀⠀⠀⠀⠀⠀⎥
⎢⡍⡡⣄⠀⠀⠀⢨⡀⠉⠥⣄⣘⢿⣷⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⢿⣷⡄⠀⡀⡀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠆⠙⢠⡀⠀⠀⠰⠉⠙⣆⣀⠀⠀⠘⢿⣷⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣿⡦⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠁⠀⠀⠹⣷⠉⠈⠀⠀⠀⠈⠉⠀⠀⠀⠉⠻⣦⡈⠀⠈⣏⠈⢳⠉⠈⠀⠀⠀⠀⠀⠈⠹⡅⠁⣏⠈⢳⠉⠈⎥
⎢⠂⠀⠀⠐⠋⣗⡖⠀⠀⠀⠀⠀⠀⠀⠀⠀⠂⠈⠻⣦⡀⠘⢰⠈⠳⡆⠀⠀⠀⠀⠀⠀⠀⠃⢦⠘⢰⡈⠓⡆⎥
⎢⡂⠀⠀⢰⡄⠘⢙⠀⠀⠀⠀⠀⠀⠀⠀⠀⡦⢤⣀⠈⢻⣶⣌⠃⠀⠛⠀⠀⠀⠀⠀⠀⢰⣶⣜⠂⠀⠃⠀⠙⎥
⎢⠆⠀⠀⠰⢤⡲⠰⠀⠀⠀⠀⠀⠀⠀⠀⠀⢦⣀⡐⠒⠦⠙⠿⣧⣄⠀⠀⠀⠀⠀⠀⠀⠈⠻⠿⠇⠀⢤⣄⠀⎥
⎢⣃⠀⠀⢘⠀⠑⢾⣀⠀⠀⠀⢀⣀⠀⠀⠀⡃⠀⠹⠦⣤⠀⠀⠙⢿⣷⣀⠀⣀⡀⠀⠀⠀⠀⠀⠀⠀⠙⢿⢷⎥
⎢⠍⠡⣄⠀⠀⠀⠨⠈⠉⢥⣄⣘⢿⣷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⢿⣷⡿⣿⡆⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⡃⠀⠈⠀⠀⠀⢘⠉⠓⣂⡀⠉⠀⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠸⣿⣯⣿⣿⡿⣿⣦⠀⠀⠀⠀⠀⠀⠀⎥
⎢⡅⠳⢀⠀⠀⠀⢨⠓⠲⡄⠉⠉⠀⠨⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉⣿⣯⣿⣿⣍⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠆⠀⠈⠳⠄⠀⠰⠀⠀⠉⠙⠒⠀⠀⠈⠋⠗⠦⠤⠀⢰⣶⣦⡀⠀⠀⠀⠀⠈⠛⠃⠙⢻⣶⣄⣶⣦⡀⠀⠀⎥
⎢⠅⠀⠀⢨⠀⠳⢨⠀⠀⠀⠀⠀⠀⠀⠀⠀⡥⢤⣈⠓⠲⠙⠿⠇⠀⠀⠀⠀⠀⠀⠀⠀⢠⣽⡿⣯⣿⣧⣤⡀⎥
⎢⠆⠀⠀⠠⣄⠀⠰⠀⠀⠀⠀⠀⠀⠀⠀⠀⢦⣀⡐⠲⠤⠀⠀⣄⣄⠀⠀⠀⠀⠀⠀⠀⠈⠻⠿⣿⡿⣯⣿⠿⎥
⎣⡃⠀⠀⢘⠈⠱⣜⠀⠀⠀⠀⠀⠀⠀⠀⠀⡃⠀⠹⠤⣄⠀⠀⠙⢿⣗⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣿⡟⢿⣷⎦

In [730]:
fnl = loadVectorNonLinear(problem, r, S)

25275-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [731]:
q = FEM.solveDisplacement(problem, [load], [supp])
S = FEM.solveStress(problem, q)
S = FEM.elementsToNodes(problem, S)

u1 = FEM.showDoFResults(problem, q, :uvec)
S1 = FEM.showDoFResults(problem, S, :s)

f = FEM.loadVector(problem, [load])
r = nodePositionVector(problem)
for i in 1:3
    Kl = stiffnessMatrixLinear(problem, r)
    Knl = stiffnessMatrixNonLinear(problem, S)
    f = FEM.loadVector(problem, [load])
    fnl = loadVectorNonLinear(problem, r, S)
    K, f = FEM.applyBoundaryConditions(problem, Kl + 0 * Knl, f - 0 * fnl, [supp])
    q = FEM.solveDisplacement(K, f)
    S = FEM.solveStress(problem, q)
    S = FEM.elementsToNodes(problem, S)
    r += q
end

r0 = nodePositionVector(problem)
u2 = FEM.showDoFResults(problem, r - r0, :uvec)
S2 = FEM.showDoFResults(problem, S, :s)

3

In [732]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [733]:
gmsh.finalize()